In [ ]:
# built-in imports
import importlib
import subprocess
import os
import re
import json

# external imports
import xarray as xr
import HydroErr
import numexpr as ne

# default environment
my_env = os.environ.copy()

# MESH-specific import
import meshflow as mf

In [ ]:
# Precompile regexes for speed/readability
_INT_RE = re.compile(r'^[-+]?\d+$')
_FLOAT_RE = re.compile(
    r"""^[-+]?(                # optional sign
        (?:\d+\.\d*|\d*\.\d+)  # something with a decimal point
        (?:[eE][-+]?\d+)?      # optional exponent
        |
        \d+[eE][-+]?\d+        # or integer with exponent (e.g. 1e6)
    )$""",
    re.X
)

def parse_numeric_string(s: str):
    """
    Try to interpret a numeric-looking string as int or float.
    Return the converted number, or the original string if not numeric.
    """
    if _INT_RE.match(s):
        # Keep as int if it fits typical Python int (Python int is unbounded anyway)
        return int(s)
    if _FLOAT_RE.match(s):
        # Anything with decimal point or exponent
        return float(s)
    return s  # not numeric-looking

def convert_numeric_strings(obj):
    """
    Recursively walk lists/dicts and convert numeric-like strings.
    """
    if isinstance(obj, dict):
        return {k: convert_numeric_strings(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [convert_numeric_strings(v) for v in obj]
    if isinstance(obj, str):
        return parse_numeric_string(obj.strip())
    return obj  # leaves int, float, bool, None, etc. untouched


def make_object_hook():
    def object_hook(d):
        for k, v in d.items():
            d[k] = convert_numeric_strings(v)  # reuse earlier function
        return d
    return object_hook

In [ ]:
# TEMPORARY
os.chdir('/Users/kasrakeshavarz/Downloads/test/etc/eval/')

In [ ]:
# there are 3 parts to model executation
input_output_path = '/Users/kasrakeshavarz/Downloads/test/' # user input to FIAT
input_date_start = '1982-01-01 00:00:00' # user input to FIAT: fiat.Calibration.dates.start
input_date_end = '2000-12-31 23:00:00' # user input to FIAT: fiat.Calibration.dates.end
input_objective_functions = {
    'QO': {
        'kge_2012': ['-1 * station_1', '-1 * station_2', '(-0.5 * station_1) + (-0.5 * station_2)'],
        'nse': ['(-0.75 * station_1) + (-0.25 * station_2)', 'sin(station_1) + cos(station_2)'],
    },
} # user input to FIAT: fiat.Calibration.objective_functions
input_output_file = 'QO_H_GRD.nc' # FIAT-MESH specific stuff
# input files from instance, names are MESH-specific defined in FIAT


# 1. pre-process
# 1.1 necessary paths (can be hard-coded)
model_instance_path = '../../model/' # can be hard-coded as it will be standard
fiat_instance_path = os.path.join(input_output_path) # can be hard-coded as it will be standard
observations_path = os.path.join(fiat_instance_path, 'etc', 'observations', 'observations.nc') # ditto

# 1.2 necessary calibration configurations
start_date = input_date_start
end_date = input_date_end

# 1.3 necessary information on objective functions
objective_functions = input_objective_functions

# 1.4 read necessary files
observations = xr.open_dataset(observations_path)

In [ ]:
# 1.5 assuring changed parameters are known to the model - FIAT+MESHFLOW?
file_paths = {
    'class_f': os.path.join(fiat_instance_path, 'etc', 'eval', 'class.json'),
    'hydro_f': os.path.join(fiat_instance_path, 'etc', 'eval', 'hydrology.json'),
    'route_f': os.path.join(fiat_instance_path, 'etc', 'eval', 'routing.json'),
    'case_f' : os.path.join(fiat_instance_path, 'etc', 'eval', 'case_entry.json'),
    'info_f' : os.path.join(fiat_instance_path, 'etc', 'eval', 'info_entry.json'),
}

files = {}

# reading jsons
for name, file_p in file_paths.items():
    with open(file_p, "r", encoding="utf-8") as f:
        files[name] = json.load(file_p, object_hook=make_object_hook())

# class
c_modified = mf.utility.render_class_template(
    class_case=files['case_f'],
    class_info=files['info_f'],
    class_grus=files['class_f']
)
# hydrology
h_modified = mf.utility.render_hydrology_template(
    routing_params=files['route_f'],
    hydrology_params=files['hydro_f'],
)
# apply changes to the MESH instance
with open(os.path.join(model_instance_path, "MESH_parameters_CLASS.ini"), "w", encoding="utf-8") as f:
    f.write(c_modified)
with open(os.path.join(model_instance_path, "MESH_parameters_hydrology.ini"), "w", encoding="utf-8") as f:
    f.write(h_modified)

# 2. run
# 2.1. temporarily change the directory to where the
#      model should run - safe thing to do
os.chdir(model_instance_path)
# 2.2. run the model
subprocess.Popen(my_command, env=my_env)

In [ ]:
# 3. create the objective function values
# 3.1. first read the time-series of obs/sim for
#      each element in the `obs` file
simulations = xr.open_dataset(os.path.join(model_instance_path, 'results', 'QO_H_GRD.nc'))

# extract names for the `observations` - can be hard-coded
station_ids = observations.subbasin.to_numpy().tolist()
station_names = observations.name.to_numpy().tolist()

In [ ]:
station_names

In [ ]:
station_ids

In [ ]:
of_values = {}

for flux, metrics in objective_functions.items():
    sims = {}
    obs = {}
    # start populating of_values
    of_values[flux] = {}
    # assign simulation results for the selected flux
    for st in station_ids:
        # sims dictionary
        sims[observations['name'].sel(subbasin=st).to_numpy().tolist()] = simulations[flux].sel(subbasin=st).to_series()
        # same for obs dictionary
        obs[observations['name'].sel(subbasin=st).to_numpy().tolist()] = observations[flux].sel(subbasin=st).to_series()
    # metric (for example, kge_2012), and ofs (list of individual objective functions
    for metric, ofs in metrics.items():
        # add elements to `of_values`
        of_values[flux][metric] = []
        # calculate the metric value
        he_metric = getattr(hydroerr_metrics, metric)
        metric_dict = {}
        for name in obs.keys():
            metric_dict[name] = he_metric(sims[name], obs[name])

        for of in ofs: # a list of objective functions
            result = ne.evaluate(expr, local_dict=metric_dict)
            of_values[flux][metric] = result
